In [38]:
import requests
import lxml.html
import re

def getHtml():
    payload = {'ymd': '20180921', 'k': '06', 'r': 1}
    r = requests.post("http://kyotei.sakura.ne.jp/bangumi2.php", data=payload)
    return r.content

def fetchRace(dom):
    data = dom.xpath("//table[@cellspacing=1]//td//a/text()")
    race = data[0]
    race = re.sub(r'\s', '', race)
    
    return {
        "race": race,
        "num": len(data) - 1
    }
    
class Race:
    def __init__(self, html):
        dom = lxml.html.fromstring(html)
        print(fetchRace(dom))

if __name__ == '__main__':
    html = getHtml()
    race = Race(html)


{'race': '1R', 'num': 6}
